# 3. OPENCV Object Detection

![](../Tutorial/images/salt-pepper.png)

<hr>

### Content
> I. [Frontal Face Detection](#face_detection)
II. [Eye Detection](#eye_detection)
III. [Smile Detection](#smile_detection)
IV. [Face-Eye-Smile Detection](#face_eye_smile_detection)
V. [Face Lines Extraction - Charcoal drawing](#charcoal)
VI. [Skin Color Detection](#skin)
<hr>


[Data Link](https://github.com/opencv/opencv/tree/master/data/haarcascades)

In [1]:
import cv2
import numpy as np
from utils import *
import imutils
from collections import deque

# I. Object Tracking

Easiest way to tracking an object is to tracking it's color

In [2]:
video_filename = ''   ## TODO
width = 600
ONLY_MAX = True
GREEN = ((29, 86, 6), (64, 255, 255))
RED = ((139, 0, 0), (255, 160, 122))
BLUE = ((110, 50, 50), (130, 255, 255))
ORANGE = ((160, 100, 47), (179, 255, 255))
YELLOW = ((10, 100, 100), (40, 255, 255))
bottom_color, top_color = BLUE
if len(video_filename) == 0:
    cam = cv2.VideoCapture(0)   # Default cam

else:
    cam = cv2.VideoCapture(video_filename)      # Read from the video file

cv2.namedWindow('Frame')
cv2.namedWindow('Mask')

while True:
    ret, frame = cam.read()     # ret is a boolean variable that returns true if the frame is available.
    if len(video_filename) > 0 and not ret: break   # EOF

    frame = imutils.resize(frame, width=width)
    hsv = cv2.GaussianBlur(frame, (25, 25), 0)
    hvs = cv2.cvtColor(hsv, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, bottom_color, top_color)

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))   # Uses dilate and erode

    mask = cv2.erode(mask, None, iterations=10)
    mask = cv2.dilate(mask, None, iterations=10)

    copy = mask.copy()

    contours = cv2.findContours(copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) [-2]

    if len(contours) > 0:
        for contour in contours:
            cmax = max(contours, key=cv2.contourArea)
            if ONLY_MAX:
                (x, y), radius = cv2.minEnclosingCircle(cmax)
            else:
                (x, y), radius = cv2.minEnclosingCircle(contour)
            if radius >= 10:
                cv2.circle(frame, (int(x), int(y)),
                           int(radius), (0, 0, 255), 4)

    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)

    key = cv2.waitKey(5) & 0xFF
    if key == 27 or key == ord('q'): break

if cam.isOpened():
    cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

# II. Adding Tail to This Object

In [2]:
video_filename = ''   ## TODO
width = 800
DOT_NUMBER = 100
GREEN = ((29, 86, 6), (64, 255, 255))
RED = ((139, 0, 0), (255, 160, 122))
BLUE = ((110, 50, 50), (130, 255, 255))
ORANGE = ((160, 100, 47), (179, 255, 255))
YELLOW = ((10, 100, 100), (40, 255, 255))
bottom_color, top_color = YELLOW
if len(video_filename) == 0:
    cam = cv2.VideoCapture(0)   # Default cam

else:
    cam = cv2.VideoCapture(video_filename)      # Read from the video file

dots = deque(maxlen=DOT_NUMBER)

cv2.namedWindow('Frame')
cv2.namedWindow('Mask')

while True:
    ret, frame = cam.read()     # ret is a boolean variable that returns true if the frame is available.
    if len(video_filename) > 0 and not ret: break   # EOF

    frame = imutils.resize(frame, width=width)
    hsv = cv2.GaussianBlur(frame, (25, 25), 0)
    hvs = cv2.cvtColor(hsv, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, bottom_color, top_color)

    # mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))   # Uses dilate and erode

    mask = cv2.erode(mask, None, iterations=8)
    mask = cv2.dilate(mask, None, iterations=8)

    copy = mask.copy()

    contours = cv2.findContours(copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) [-2]

    center = None
    if len(contours) > 0:
        cmax = max(contours, key=cv2.contourArea)
        for contour in contours:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            moments = cv2.moments(cmax)
            center = int(moments['m10'] / moments['m00']), int(moments['m01'] / moments['m00'])
            if radius >= 30:
                cv2.circle(frame, (int(x), int(y)),
                           int(radius), (0, 0, 255), 4)
            dots.appendleft(center)
            for i in range(1, len(dots)):
                if dots[i] and dots[i - 1]:
                    line_cons = 2
                    cv2.line(frame, dots[i - 1], dots[i], (0, 255, 255), line_cons)
                    
    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)

    key = cv2.waitKey(5) & 0xFF
    if key == 27 or key == ord('q'): break

if cam.isOpened():
    cam.release()
cv2.destroyAllWindows()

# 3.